In [1]:

import pandas as pd
import numpy as np
import math
from collections import Counter

In [2]:
sensor_data = pd.DataFrame.from_csv("../../data/database/sensor_data.csv")
Counter(sensor_data.TagName)

Counter({'Start1': 213100,
         'Start2': 213808,
         'Tag0': 352461,
         'Tag1': 272234,
         'Tag2': 377420,
         'Tag3': 398370,
         'Tag4': 428642,
         'Tag5': 435086,
         'Tag6': 358288,
         'Tag7': 315266,
         'Tag8': 431390,
         'Tag9': 381880})

In [3]:

sensor_data_acc = sensor_data[sensor_data.SENSORTYPE == 1]
sensor_data_acc_number_tag = sensor_data_acc[~((sensor_data_acc.TagName == 'Tag0') | (sensor_data_acc.TagName == 'Start1') | (sensor_data_acc.TagName == 'Start2'))]

In [4]:
Counter(sensor_data_acc_number_tag.TagName)

Counter({'Tag1': 41185,
         'Tag2': 57227,
         'Tag3': 60499,
         'Tag4': 65089,
         'Tag5': 66044,
         'Tag6': 54351,
         'Tag7': 47811,
         'Tag8': 65394,
         'Tag9': 57911})

In [5]:
# tag_id_groupby = sensor_data_acc_tag12.groupby(['TagName', 'tester_id'])
tag_id_groupby = sensor_data_acc_number_tag.groupby(['TagName', 'tester_id'])
tag_id_dict = tag_id_groupby.groups
y = list(tag_id_dict.keys())
y = [i[0] for i in y]
y = np.array(y)

In [7]:
# there will be 9 frames, frame0 to frame8
# There will be 10 segments
N_frame_no = 9 

test_percent = 0.1 # 10% samples are used for testing

In [8]:
def gesture_features(accs):
    Ls = math.floor(len(accs)/ (N_frame_no + 1))
    segments = None
    for i in range(N_frame_no + 1):
        if segments is None:
            segments = np.array([accs[i*Ls:(i+1)*Ls]])
        else:
            segments = np.append(segments, np.array([accs[i*Ls:(i+1)*Ls]]), axis=0)

    frames = None
    for i in range(N_frame_no):
        cur_frame = segments[i:i+2]
        cur_frame = cur_frame.reshape((cur_frame.shape[0]*cur_frame.shape[1],cur_frame.shape[2]))
        if frames is None:
            frames = np.array([cur_frame])
        else:
            frames = np.append(frames, np.array([cur_frame]), axis = 0)
    return np.array([frame_features(f) for f in frames]).reshape(-1)

In [9]:
def frame_features(cur_frame):
    Ls = len(cur_frame)*2
    
    dft_cur_frame = np.fft.fftn(cur_frame)
    
    mean_cur_frame = dft_cur_frame[0]

    energy_cur_frame=[]
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        for i in range(1,Ls*2):
            T_sum += math.pow(abs(dft_cur_frame[i,T]),2)

        energy_cur_frame.append(T_sum / abs(Ls*2-1))
    energy_cur_frame = np.array(energy_cur_frame)
    
    entropy_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        entropy_sum = 0
        for i in range(1,Ls*2):
            T_sum += abs(dft_cur_frame[i,T])

        for m in range(1,Ls*2):
            p_m_T = abs(dft_cur_frame[m,T])/T_sum
            entropy_sum += p_m_T*math.log(1/p_m_T)
        entropy_cur_frame.append(entropy_sum)
    entropy_cur_frame = np.array(entropy_cur_frame)
    
    std_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        std_cur_frame.append(np.std(cur_frame))
    std_cur_frame = np.array(std_cur_frame)
    
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    # correlation
    
    return np.concatenate((mean_cur_frame,energy_cur_frame,std_cur_frame,entropy_cur_frame))

In [ ]:
X>shape

In [10]:
X = []
for key in list(tag_id_dict.keys()):
    X.append(gesture_features(tag_id_groupby.get_group(key)[['VALUES1', 'VALUES2', 'VALUES3']].values).reshape(-1))
X = np.array(X)

In [11]:
X.shape

(963, 108)

In [12]:
from sklearn.svm import SVC
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [13]:
idx_list = list(range(len(X)))
np.random.shuffle(idx_list)
train_idx = idx_list[:math.floor(len(X) * (1- test_percent))]
test_idx = idx_list[math.floor(len(X) * (1- test_percent)):]

In [14]:
# y[y == 'Tag2'] = 2
# y[y == 'Tag1'] = 1

train_x = X[train_idx]
train_y = y[train_idx]
test_x = X[test_idx]
test_y = y[test_idx]

In [15]:
clf4 = SVC(kernel='poly', C=1, degree=4, verbose = True)
clf4.fit(train_x, train_y) 
joblib.dump(clf4, './09 Feb SVC degree 4.pkl') 
print(classification_report(test_y, clf4.predict(test_x))

[LibSVM]

/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


['./09 Feb SVC.pkl']

In [23]:
clf3 = SVC(kernel='poly', C=1, degree=3, verbose = True)
clf3.fit(train_x, train_y) 
joblib.dump(clf3, './09 Feb SVC degree3.pkl') 
print(classification_report(test_y, clf3.predict(test_x)))

[LibSVM]             precision    recall  f1-score   support

       Tag1       0.56      0.62      0.59         8
       Tag2       0.50      0.57      0.53        14
       Tag3       0.67      0.43      0.52        14
       Tag4       0.82      0.75      0.78        12
       Tag5       0.45      0.45      0.45        11
       Tag6       0.75      0.55      0.63        11
       Tag7       0.73      0.67      0.70        12
       Tag8       0.42      0.71      0.53         7
       Tag9       0.50      0.62      0.56         8

avg / total       0.61      0.59      0.59        97



/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


In [22]:
clf2 = SVC(kernel='poly', C=1, degree=2, verbose = True)
clf2.fit(train_x, train_y) 
joblib.dump(clf2, './09 Feb SVC degree2.pkl') 
print(classification_report(test_y, clf2.predict(test_x)))

[LibSVM]             precision    recall  f1-score   support

       Tag1       0.56      0.62      0.59         8
       Tag2       0.60      0.64      0.62        14
       Tag3       0.82      0.64      0.72        14
       Tag4       0.75      0.75      0.75        12
       Tag5       0.42      0.45      0.43        11
       Tag6       0.73      0.73      0.73        11
       Tag7       0.78      0.58      0.67        12
       Tag8       0.62      0.71      0.67         7
       Tag9       0.60      0.75      0.67         8

avg / total       0.66      0.65      0.65        97



/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


In [25]:
clf1 = SVC(kernel='poly', C=1, degree=1, verbose = False)
clf1.fit(train_x, train_y) 
joblib.dump(clf1, './09 Feb SVC degree1.pkl') 
print(classification_report(test_y, clf1.predict(test_x)))

/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


NameError: name 'clf24predict' is not defined

             precision    recall  f1-score   support

       Tag1       0.50      0.50      0.50         8
       Tag2       0.62      0.71      0.67        14
       Tag3       0.70      0.50      0.58        14
       Tag4       0.73      0.67      0.70        12
       Tag5       0.67      0.73      0.70        11
       Tag6       0.67      0.73      0.70        11
       Tag7       0.90      0.75      0.82        12
       Tag8       0.75      0.86      0.80         7
       Tag9       0.50      0.62      0.56         8

avg / total       0.68      0.67      0.67        97



/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


In [ ]:
clf5 = SVC(kernel='poly', C=1, degree=5, verbose = False)
clf5.fit(train_x, train_y) 
joblib.dump(clf5, './09 Feb SVC degree5.pkl') 
print(classification_report(test_y, clf5.predict(test_x)))

/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
